## Setup

In [261]:
from dotenv import load_dotenv
from utils import chat_interface
import os
import math
from datetime import datetime, timezone

from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.tools import tool
from langgraph.graph.message import MessagesState
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from pydantic import BaseModel, Field, ConfigDict
from typing import Dict, Any, List, Annotated, Literal, Tuple, Optional
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

### Parameters

In [27]:
num_retrieval = 5

### Setup LLMs for different Technical Support Level

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# llm_base_url = "https://openai.vocareum.com/v1"
llm_base_url = "https://api.openai.com/v1"


llm_small = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

llm_medium = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

llm_large = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

embeddings_fn = OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url=llm_base_url,
    api_key=OPENAI_API_KEY,
)

### Create Vector Database Connection for RAG

In [29]:
chromadb_directory = "vectorstore"
collection_name = "knowledge_vecotr_store"
vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings_fn,
    persist_directory=chromadb_directory,
)

## Agent State Management

State Memory

In [ ]:
class TicketState(MessagesState):
    """State for the IT ticket workflow."""
    query: HumanMessage
    current_answers: AIMessage
    # retrieved documents
    documents: List[Document]
    # routing signals
    priority: Literal['normal', 'urgent', 'vip'] = 'normal'
    intent: Literal['how_to', 'troubleshooting_basic', 'status_check', 
                    'security', 'billing_payment', 'outage', 'others'] = 'others'
    retrieval_score: float = 0.0
    answer_confidence: float = 0.0
    sentiment: float = 0.0
    # control/status
    assigned_group: Literal['L1', 'L2', 'L3', 'human'] = 'L1'
    eval_status: Literal['resolved', 'needs_clarification', 'escalate'] = 'needs_clarification'
    turns_at_level: int = 0
    escalation_reason: Optional[Literal['low_confidence',
                                        'high_impact_or_sensitive',
                                        'unresolved_or_negative',
                                        'sla_breach']] = None
    # SLA/aging
    level_entered_at: datetime = datetime.now(timezone.utc)
    sla_minutes: int = 3  # example per level

Standardized Output for Subagent

In [ ]:
class IntentOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    intent: Annotated[Literal['how_to','troubleshooting_basic','status_check','security','refund_high','outage','others'], 
                      Field(description="The intent of the IT ticket")]
    
class SentimentOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    sentiment: Annotated[float, Field(description="The user's sentiment",
                                      ge=-1.0,
                                      le=1.0)]

class ConfidenceOutput(BaseModel):
    """Standardized output to ensure it is from the possible values"""
    confidencet: Annotated[float, Field(description="The confidence of answer toward the query",
                                      ge=0.0,
                                      le=1.0)]

# ChromaDB Similarity Search Output Structure        
class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    metadata: Dict[str, Any]

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    """Standardized output"""
    intent: Annotated[Literal['how_to','troubleshooting_basic','status_check','security','refund_high','outage','others'], 
                      Field(description="The intent of the IT ticket")]
    sentiment: Annotated[float, Field(description="The user's sentiment",
                                      ge=-1.0,
                                      le=1.0)]


# Agent Tools

In [248]:
@tool
def knowledge_retrival(query: str) -> List[dict]:
    """
    Evaluate whether the knowledge base has enough content to address the query and return the relevant content indices.

    Args:
        query (str): The user query string
    
    Returns:
        retrival (List[tuple[Document,float]]): Response from the ChromaDB database for the retrieved documents and the corresponding
                                                distance measure.
    """
    results = vector_store.similarity_search_with_score(
        query=query,
        k=num_retrieval,
    )

    # Convert the output to recognizable types for Pydantic to work
    retrival = []
    for doc, score in results:
        retrival.append({
            "document": {
                "id": int(doc.id),
                "page_content": doc.page_content,
                "tags": doc.metadata.get("tags","")
            },
            "score": score
        })
    return retrival

def run_retrieval(query: str):
    return {"query": query, "retrival": knowledge_retrival.func(query)}


In [ ]:
@tool
def intent_check(query: str) -> str:
    """
    Evaluate the intent of the query for routing decision.

    Args:
        query (str): The user query string
    
    Returns:
        intent (str): One of ['how_to', 'troubleshooting_basic', 'status_check',
                              'security', 'billing_payment', 'outage', 'others']
    """
    
    prompt_template = """
        You are an expert evaluating the intent of IT ticket submitted by users and you are suppose to
        classify it as one of the following intent: ['how_to', 'troubleshooting_basic', 'status_check',
        'security', 'billing_payment', 'outage', 'others'].

        Output a single one of the above intent in the list, do not provide any explanation or description
        to your answer.

        Here is the user query:
        {query}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})
    response = llm_medium.with_structured_output(IntentOutput).invoke(prompt)
    intent = response.intent
    return intent
    

In [62]:
@tool
def sentiment_analyzer(query: str) -> float:
    """
    Evaluate the sentiment of the input query and output a score between -1 to 1.

    Args:
        query (str): The user query string
    
    Returns:
        sentiment (float): Score between -1 to 1, where -1 is upset/angry/negative where 1 is happy/optimisitic/positive
    """
    prompt_template = """
        You are an expert in marketing and assessing the sentiment of the customer.  You need to provide a score between
        -1 to 1 where -1 is upset/angry/negative where 1 is happy/optimisitic/positive.

        Output a single number, do not provide any explanation or description to your answer.

        Here is the user query:
        {query}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})
    response = llm_medium.with_structured_output(SentimentOutput).invoke(prompt)
    sentiment = response.sentiment
    return sentiment

In [63]:
@tool
def confidence_evaluator(query: str, answer: str) -> float:
    """
    Evaluate the confidence of the answer towards answering the query by providing a score between 0 to 1.

    Args:
        query (str): The user query string
        answer (str): The answer provided by the IT support
    
    Returns:
        confidence (float): Score between 0 to 1, where 0 corresponse to an answer where it is completely irrelevant to
        the querey, where 1 is spot on and it solves the user's query.
    """
    prompt_template = """
        You are an expert in quality control to evaluate whether the support team had been doing a good job for providing
        the relevant response to the user.  You need to provide a score between 0 to 1, where 0 corresponse to an answer 
        where it is completely irrelevant to the querey, where 1 is spot on and it solves the user's query.

        Here is the user query:
        {query}

        Here is the support team's response:
        {answer}
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query","answer"],
        ).invoke({"query": query, "answer": answer})
    response = llm_medium.with_structured_output(ConfidenceOutput).invoke(prompt)
    confidence = response.confidence
    return confidence

In [263]:
llm_initial_evaluator = llm_medium.bind_tools([knowledge_retrival,intent_check,sentiment_analyzer])
llm_answer_evaluator = llm_medium.bind_tools([confidence_evaluator])

## Agents

In [ ]:
def query_assessment_agent(state: TicketState):
    query = state["query"]
    # Obtain Retrival Score and Documents
    retrivals = vector_store.similarity_search_with_score(
        query=query,
        k=num_retrieval,
        )

    documents = [ retrival[0] for retrival in retrivals ]
    similarity = [ 1-(retrival[1]/math.sqrt(2)) for retrival in retrivals ]  # ChromaDB uses L2 distance, it requires to perform this operation to convert to similarity
    max_similarity_score = max(similarity)

    # Intent Checking and Sentiment Analysis
    llm_initial_evaluator = llm_medium.bind_tools([intent_check,sentiment_analyzer])
    prompt_template = """
        You are a IT ticket evaluation agent, given the user query:
        {query}
        Use the following tools to provide insights for the user query:
            (1) Use `intent_check` to identify the "intent" attribute of the given query
            (2) Use `sentiment_analyzer` to estimate the "sentiment" of the given query
        """
    prompt = PromptTemplate(
        template = prompt_template,
        input_variables = ["query"],
        ).invoke({"query": query})
    response = llm_initial_evaluator.with_structured_output(QueryAttributeOutput).invoke(prompt)
    
    ai_message = AIMessage(content=f"intent: {response.intent_check}, retrival_score: {max_similarity_score}, sentiment: {response.sentiment}")

    return {"intent": response.intent_check,
            "retrival_score": max_similarity_score,
            "sentiment": response.sentiment,
            "documents": documents,
            "messages": ai_message}

In [ ]:
L1_TOPICS = {"how_to","troubleshooting_basic","status_check"}
L2_TOPICS = {"others","billing_payment"}
L3_TOPICS = {"outage","security"}

In [ ]:
def route_agent(state: TicketState):
    score = state["retrieval_score"]
    intent = state["intent"]
    if score < 0.4 or intent in L3_TOPICS:
        return "L3"
    if intent in L2_TOPICS:
        return "L2"
    return "L1"

In [ ]:
def escalation_agent(state: TicketState):
    # Evaluate answer confidence
    

In [ ]:
def l1_agent(state: TicketState):
    question = state["question"]
    documents = state["documents"]
    docs_content = "\n\n".join(doc.page_content for doc in documents)

    template = ChatPromptTemplate([
        ("system", "You are an assistant for question-answering tasks."),
        ("human", """
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know.
            Use three sentences maximum and keep the answer concise.
            # Question: -> {question}
            # Context: -> {context}
            # Answer:
            """
            ),
        ])

    messages = template.invoke(
        {"context": docs_content, "question": question}
        ).to_messages()

    ai_message = llm_small.invoke(messages)

    return {"messages": ai_message,
            "current_answer": ai_message}


## Workflow

In [ ]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

In [ ]:
chat_interface(orchestrator, "1")

In [ ]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

---

# Testing Code

In [254]:
@tool
def knowledge_retrival(query: str) -> List[dict]:
    """
    Evaluate whether the knowledge base has enough content to address the query and return the relevant content indices.

    Args:
        query (str): The user query string
    
    Returns:
        retrival (List[tuple[Document,float]]): Response from the ChromaDB database for the retrieved documents and the corresponding
                                                distance measure.
    """
    results = vector_store.similarity_search_with_score(
        query=query,
        k=num_retrieval,
    )

    # Convert the output to recognizable types for Pydantic to work
    retrival = []
    for doc, score in results:
        retrival.append({
            "document": {
                "id": int(doc.id),
                "page_content": doc.page_content,
                "tags": doc.metadata.get("tags","")
            },
            "score": score
        })
    return retrival

def run_retrieval(query: str):
    return {"query": query, "retrival": knowledge_retrival.func(query)}

class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    tags: str

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    retrival: List[SearchResultItem] = Field(min_length=num_retrieval, description="List of retrieved document and score pairs.")

query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."

prompt = ChatPromptTemplate.from_template(
    "Return the provided retrievals verbatim in the output schema.\nRetrievals:\n{retrival}"
    )

pipeline = (
    RunnableLambda(lambda q: {"retrival": knowledge_retrival.func(q)})
    | prompt
    | llm_medium.with_structured_output(QueryAttributeOutput)
    )
retrivals = pipeline.invoke(query)


In [256]:
retrivals.retrival

[SearchResultItem(document=VectorDocument(id=19, page_content='How to Handle Billing Discrepancies: For billing issues:\n\n- Review your billing history in the CultPass app\n- Contact support if discrepancies are found\n- Provide transaction details for faster resolution\n\n**Suggested phrasing:**\n"If you notice any billing discrepancies, review your history and contact support with transaction details.\'', tags='billing, escalation, support'), score=1.0416605472564697),
 SearchResultItem(document=VectorDocument(id=5, page_content='Understanding CultPass Pricing: CultPass offers flexible pricing options:\n\n- Monthly subscription: Access to 4 experiences per month\n- Annual subscription: Discounted rate for a year-long access\n- Additional fees may apply for premium events\n\n**Suggested phrasing:**\n"Explore our flexible pricing plans, including monthly and annual subscriptions. Note that some premium events may incur additional fees.\'', tags='pricing, subscription, benefits'), scor

In [ ]:

query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."

retrivals = vector_store.similarity_search_with_score(
    query=query,
    k=num_retrieval,
    )
similarity = [ 1-(retrival[1]/math.sqrt(2)) for retrival in retrivals ]  # ChromaDB uses L2 distance, it requires to perform this operation to convert to similarity
max_similarity_score = max(similarity)

# Intent Checking and Sentiment Analysis
llm_initial_evaluator = llm_medium.bind_tools([intent_check,sentiment_analyzer])
prompt_template = """
    You are a IT ticket evaluation agent, given the user query:
    {query}

    Use the following tools to provide insights for the user query:
        (1) Use `intent_check` to identify the "intent" attribute of the given query
        (2) Use `sentiment_analyzer` to estimate the "sentiment" of the given query
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})
response = llm_initial_evaluator.with_structured_output(QueryAttributeOutput).invoke(prompt)

In [230]:
retrivals

[(Document(id='19', metadata={'tags': 'billing, escalation, support'}, page_content='How to Handle Billing Discrepancies: For billing issues:\n\n- Review your billing history in the CultPass app\n- Contact support if discrepancies are found\n- Provide transaction details for faster resolution\n\n**Suggested phrasing:**\n"If you notice any billing discrepancies, review your history and contact support with transaction details."'),
  1.0416462421417236),
 (Document(id='5', metadata={'tags': 'pricing, subscription, benefits'}, page_content='Understanding CultPass Pricing: CultPass offers flexible pricing options:\n\n- Monthly subscription: Access to 4 experiences per month\n- Annual subscription: Discounted rate for a year-long access\n- Additional fees may apply for premium events\n\n**Suggested phrasing:**\n"Explore our flexible pricing plans, including monthly and annual subscriptions. Note that some premium events may incur additional fees."'),
  1.2668156623840332),
 (Document(id='14

In [238]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."

class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    tags: str

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    retrival: Annotated[List[SearchResultItem], Field(description="List of retrieved document and score pairs.")]

results = vector_store.similarity_search_with_score(
    query=query,
    k=num_retrieval,
)

# Convert the output to recognizable types for Pydantic to work
retrival = []
for doc, score in results:
    retrival.append({
        "document": {
            "id": int(doc.id),
            "page_content": doc.page_content,
            "tags": doc.metadata.get("tags")
        },
        "score": score
    })

QueryAttributeOutput(retrival=retrival)

QueryAttributeOutput(retrival=[SearchResultItem(document=VectorDocument(id=19, page_content='How to Handle Billing Discrepancies: For billing issues:\n\n- Review your billing history in the CultPass app\n- Contact support if discrepancies are found\n- Provide transaction details for faster resolution\n\n**Suggested phrasing:**\n"If you notice any billing discrepancies, review your history and contact support with transaction details."', tags='billing, escalation, support'), score=1.0415587425231934), SearchResultItem(document=VectorDocument(id=5, page_content='Understanding CultPass Pricing: CultPass offers flexible pricing options:\n\n- Monthly subscription: Access to 4 experiences per month\n- Annual subscription: Discounted rate for a year-long access\n- Additional fees may apply for premium events\n\n**Suggested phrasing:**\n"Explore our flexible pricing plans, including monthly and annual subscriptions. Note that some premium events may incur additional fees."', tags='pricing, sub

In [239]:
len(retrival)

5

In [158]:
QueryAttributeOutput(retrival=retrival)

QueryAttributeOutput(retrival=[SearchResultItem(document=VectorDocument(id=19, page_content='How to Handle Billing Discrepancies: For billing issues:\n\n- Review your billing history in the CultPass app\n- Contact support if discrepancies are found\n- Provide transaction details for faster resolution\n\n**Suggested phrasing:**\n"If you notice any billing discrepancies, review your history and contact support with transaction details."', metadata={'tags': 'billing, escalation, support'}), score=1.0406265258789062), SearchResultItem(document=VectorDocument(id=5, page_content='Understanding CultPass Pricing: CultPass offers flexible pricing options:\n\n- Monthly subscription: Access to 4 experiences per month\n- Annual subscription: Discounted rate for a year-long access\n- Additional fees may apply for premium events\n\n**Suggested phrasing:**\n"Explore our flexible pricing plans, including monthly and annual subscriptions. Note that some premium events may incur additional fees."', meta

In [153]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
prompt_template = """
    You are an expert evaluating the intent of IT ticket submitted by users and you are suppose to
    classify it as one of the following intent: ['how_to', 'troubleshooting_basic', 'status_check',
    'security', 'refund_high', 'outage', 'others'].

    Output a single one of the above intent in the list, do not provide any explanation or description
    to your answer.

    Here is the user query:
    {query}
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})
response = llm_medium.with_structured_output(IntentOutput).invoke(prompt)


In [54]:
query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
query = "You are so useless!"
prompt_template = """
    You are an expert in marketing and assessing the sentiment of the customer.  You need to provide a score between
    -1 to 1 where -1 is upset/angry/negative where 1 is happy/optimisitic/positive.

    Output a single number, do not provide any explanation or description to your answer.

    Here is the user query:
    {query}
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})
response = llm_medium.with_structured_output(SentimentOutput).invoke(prompt)
sentiment = response.sentiment
sentiment

-1.0

In [240]:
@tool
def knowledge_retrival(query: str):
    """
    Evaluate whether the knowledge base has enough content to address the query and return the relevant content indices.

    Args:
        query (str): The user query string
    
    Returns:
        retrival (List[tuple[Document,float]]): Response from the ChromaDB database for the retrieved documents and the corresponding
                                                distance measure.
    """
    results = vector_store.similarity_search_with_score(
        query=query,
        k=num_retrieval,
    )

    # Convert the output to recognizable types for Pydantic to work
    retrival = []
    for doc, score in results:
        safe_metadata = json.loads(json.dumps(doc.metadata, default=str))
        retrival.append({
            "document": {
                "id": int(doc.id),
                "page_content": doc.page_content,
                "tags": doc.metadata.get("tags","")
            },
            "score": score
        })
    return retrival

class VectorDocument(BaseModel):
    """The contents of a retrieved LangChain Document."""
    id: int
    page_content: str
    tags: str

class SearchResultItem(BaseModel):
    """Represents the tuple (Document, score) as a JSON object."""
    document: VectorDocument = Field(description="The document content and metadata.")
    score: float = Field(description="The L2 distance.")

class QueryAttributeOutput(BaseModel):
    retrival: List[SearchResultItem] = Field(min_length=num_retrieval, description="List of retrieved document and score pairs.")

query = "Hi, I have a question regarding the billing for the events I attended last month. There seems to be a discrepancy in the charges, and I would like to understand the breakdown of the costs. Could you please provide a detailed billing statement? Thank you."
llm_initial_evaluator = llm_medium.bind_tools([knowledge_retrival])
# Evaluate Retrival Score, Intent, Sentiment of the Query
prompt_template = """
    You are a IT ticket evaluation agent, given the user query:
    {query}

    Use `knowledge_retrival` to extract "retrival" attribute from the ChromaDB knowledge database
    """
prompt = PromptTemplate(
    template = prompt_template,
    input_variables = ["query"],
    ).invoke({"query": query})

response = llm_initial_evaluator.with_structured_output(QueryAttributeOutput).invoke(prompt)

In [241]:
len(response.retrival)

5

In [242]:
response.retrival

[SearchResultItem(document=VectorDocument(id=1, page_content='Billing Statement for Events Attended in September 2023:\n\n1. Event Name: Tech Conference 2023\n   Date: September 10, 2023\n   Charges: $150.00\n   Description: Full access to all sessions and networking events.\n\n2. Event Name: Workshop on AI\n   Date: September 15, 2023\n   Charges: $75.00\n   Description: Hands-on workshop with expert speakers.\n\n3. Event Name: Networking Dinner\n   Date: September 20, 2023\n   Charges: $50.00\n   Description: Dinner with keynote speakers and industry leaders.\n\nTotal Charges: $275.00\n\nIf you have any further questions or need additional details, please let us know!', tags='billing, events, September 2023'), score=0.95),
 SearchResultItem(document=VectorDocument(id=2, page_content='To address billing discrepancies, please review the following:\n\n- Ensure that you have registered for the events you attended.\n- Check for any promotional codes or discounts that may apply.\n- Verify 